<a href="https://colab.research.google.com/github/jibbals/supersat/blob/master/Batch_Super_Res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Mounted at /content/gdrive


In [0]:
  !curl -s https://course.fast.ai/setup/colab | bash


Updating fastai...
Done.


In [0]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *
from torchvision.models import vgg16_bn

In [0]:
path = base_dir + 'All Images'

path_hr = Path(path + '/SH')
path_mr = Path(path + '/SM')
path_lr = Path(path + '/SL')
path_land_sat = Path(base_dir +'Test Land Sat bunch/LS 160')
path_empty = Path(path +'/empty')

In [0]:
il = ImageList.from_folder(path_empty)

In [0]:
# def resize_one(fn, i, path, size):
#     dest = path/fn.relative_to(path_hr)
#     dest.parent.mkdir(parents=True, exist_ok=True)
#     img = PIL.Image.open(fn)
#     targ_sz = resize_to(img, size, use_min=True)
#     img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
#     img.save(dest, quality=60)

In [0]:
#create smaller image sets the first time this nb is run
# sets = [(path_lr, 96), (path_mr, 256)]
# for p,size in sets:
#     if not p.exists(): 
#         print(f"resizing to {size} into {p}")
#         parallel(partial(resize_one, path=p, size=size), il.items)

In [0]:
bs,size=64,128 #32,128
arch = models.resnet34

src = ImageImageList.from_folder(path_empty).split_by_rand_pct(0.5, seed=42)

In [0]:
def get_data(bs,size):#was path_hr
    data = (src.label_from_func(lambda x: path_empty/x.name)
           .transform(get_transforms(max_zoom=2.), size=size, tfm_y=True)
           .databunch(bs=bs).normalize(imagenet_stats, do_y=True))

    data.c = 3
    return data


# def get_data(bs,size):
#     data = (src.label_from_func(lambda x: path_empty/x.name). #was path_hr
#            .transform(get_transforms(max_zoom=2.), size=size, tfm_y=True)
#            .databunch(bs=bs).normalize(imagenet_stats, do_y=True))

#     data.c = 3
#     return data

In [0]:
data = get_data(bs,size)
data

You can deactivate this warning by passing `no_check=True`.


/usr/local/lib/python3.6/dist-packages/fastai/basic_data.py:248: UserWarning: Your training dataloader is empty, you have only 1 items in your training set.
                 Your batch size is 64, you should lower it.
  Your batch size is {self.train_dl.batch_size}, you should lower it.""")


ImageDataBunch;

Train: LabelList (1 items)
x: ImageImageList
Image (3, 128, 128)
y: ImageList
Image (3, 128, 128)
Path: /content/gdrive/My Drive/fastai-v3/All Images/empty;

Valid: LabelList (1 items)
x: ImageImageList
Image (3, 128, 128)
y: ImageList
Image (3, 128, 128)
Path: /content/gdrive/My Drive/fastai-v3/All Images/empty;

Test: None

In [0]:
t = data.valid_ds[0][1].data
t = torch.stack([t,t])

In [0]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [0]:
gram_matrix(t)
base_loss = F.l1_loss

In [0]:
# copy_to = '/root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth'
# copy_from = base_dir + 'vgg16_bn-6c64b313.pth'
# print(copy_from)
# shutil.copyfile(copy_from, copy_to) 

In [0]:
vgg_m = vgg16_bn(True).features.cuda().eval()
requires_grad(vgg_m, False)

In [0]:
blocks = [i-1 for i,o in enumerate(children(vgg_m)) if isinstance(o,nn.MaxPool2d)]
blocks, [vgg_m[i] for i in blocks]

([5, 12, 22, 32, 42],
 [ReLU(inplace=True),
  ReLU(inplace=True),
  ReLU(inplace=True),
  ReLU(inplace=True),
  ReLU(inplace=True)])

In [0]:
class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): self.hooks.remove()

In [0]:
feat_loss = FeatureLoss(vgg_m, blocks[2:5], [5,15,2])

In [0]:
# copy_to = '/root/.cache/torch/checkpoints/resnet34-333f7ec4.pth'
# copy_from = base_dir + 'resnet34-333f7ec4.pth'
# print(copy_from)
# shutil.copyfile(copy_from, copy_to) 

In [0]:
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, loss_func=feat_loss, callback_fns=LossMetrics,blur=True, norm_type=NormType.Weight)
gc.collect();

In [0]:
free = gpu_mem_get_free_no_cache()
# the max size of the test image depends on the available GPU RAM 
if free > 8000: size=(400, 400) # >  8GB RAM
else:           size=( 400, 400) # <= 8GB RAM
print(f"using size={size}, have {free}MB of GPU RAM free")

using size=(400, 400), have 15337MB of GPU RAM free


In [0]:
learn = unet_learner(data, arch, loss_func=F.l1_loss, blur=True, norm_type=NormType.Weight)

In [0]:
data_mr = (ImageImageList.from_folder(path_land_sat).split_by_rand_pct(1, seed=42)
          .label_from_func(lambda x: path_land_sat/x.name)
          .transform(get_transforms(), size=size, tfm_y=True)
          .databunch(bs=1).normalize(imagenet_stats, do_y=True))
data_mr.c = 3

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:454: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")


In [0]:
#learn = unet_learner(data_mr, arch, loss_func=F.l1_loss, blur=True, norm_type=NormType.Weight)

In [0]:
super_res_model = '/content/gdrive/My Drive/fastai-v3/Super Res v2.2'
learn.load(super_res_model)
print('p')

p


In [0]:
#make folder for super res images 
new_folder = os.path.dirname(path_land_sat)
super_res_folder = 'LS 160 HR'
sr_dir = os.path.join(new_folder,super_res_folder)
print(sr_dir)
Path(sr_dir).mkdir(parents=True, exist_ok=True)
#import glob
#ls_path_str = str(path_land_sat)
#list_of_LS_img = (glob.glob(ls_path_str+"/*.png"))

/content/gdrive/My Drive/fastai-v3/Test Land Sat bunch/LS 160 HR


In [0]:
import imageio
#print(SR_image_dir)



In [0]:
labels = data_mr.valid_dl.dataset.items
labels_list = labels.tolist()
images_list = list(map(str,(labels_list)))
#images_list

In [0]:
import cv2

learn.data = data_mr
index = 0
for lr_img in data_mr.valid_ds.x.items:
  fn = data_mr.valid_ds.x.items[index]; 
  
  img = open_image(fn); img.shape
  p,img_hr,b = learn.predict(img)


  #show_image(img, figsize=(18,15), interpolation='nearest');

  #print(index)

  file_name = os.path.basename(images_list[index])

  sr_dirP = (Path(sr_dir))

  p.save(sr_dirP/file_name)




  # img_hr_np = img_hr.numpy()

  # img_ht_np2 = np.moveaxis(img_hr_np,0,2)

  # SR_image_name = file_name

  # SR_image_dir = os.path.join(sr_dir,SR_image_name)
  # #print(img_ht_np2.min(),img_ht_np2.max())

  # save(SR_image_dir/names[i].name)

  #imageio.imwrite(SR_image_dir,img_ht_np2)
  

  index+=1
  #print(index)

In [0]:

  # img = cv2.imread(str(fn))
  # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # img_0_max = img[0].max()
  # img_0_min = img[0].min()
  # img_1_max = img[1].max()
  # img_1_min = img[1].min()
  # img_2_max = img[2].max()
  # img_2_min = img[2].min()

  # print(img_ht_np2.shape)
  # img_ht_np2 = np.moveaxis(img_hr_np,2,1)


  # img_ht_np2_0_max = img_ht_np2[0].max()
  # img_ht_np2_0_min = img_ht_np2[0].min()
  # img_ht_np2_1_max = img_ht_np2[1].max()
  # img_ht_np2_1_min = img_ht_np2[1].min()
  # img_ht_np2_2_max = img_ht_np2[2].max()
  # img_ht_np2_2_min = img_ht_np2[2].min()


  # img_ht_np2_0 = ((abs(img_ht_np2_0_min) + img_ht_np2[0])*img_0_max/(img_ht_np2_0_max+abs(img_ht_np2_0_min)))
  # img_ht_np2_1 = ((abs(img_ht_np2_1_min) + img_ht_np2[1])*img_1_max/(img_ht_np2_1_max+abs(img_ht_np2_1_min)))
  # img_ht_np2_2 = ((abs(img_ht_np2_2_min) + img_ht_np2[2])*img_2_max/(img_ht_np2_2_max+abs(img_ht_np2_2_min)))


  # rebuilt_image =np.stack((img_ht_np2_0, img_ht_np2_1,img_ht_np2_2), axis = 2).astype(np.uint8)
  # #print(rebuilt_image)

#fn = data_mr.valid_ds.x.items[index]; 
# img = open_image(fn); img.shape
# print(img.data)
#p,img_hr,b = learn.predict(img)
fn

PosixPath('/content/gdrive/My Drive/fastai-v3/Test Land Sat bunch/Col_ls5/20011_ga_ls5t_ard_3_0075.png')

In [0]:




#rebuilt_image.shape
#print(img_0_max)
#print(img_ht_np2_0.max())
# print(img_0_min)
# print(img_ht_np2_0.min())
# print(img_1_min)
# print(img_ht_np2_1.min())
# print(img_2_min)
# print(img_ht_np2_2.min())

# print(img_0_max)
# print(img_ht_np2_0.max())
# print(img_1_max)
# print(img_ht_np2_1.max())
# print(img_2_max)
# print(img_ht_np2_2.max())



#print(img_ht_np2_0.min())




#print(img_ht_np2)
#show_image(img_ht_np2, figsize=(18,15), interpolation='nearest');


(3, 400, 400)
[[[55.343575 51.20875  51.307434 52.86357  ... 49.069096 49.5334   50.87641  50.79513 ]
  [52.715816 51.736404 50.73166  51.71347  ... 48.351105 48.5204   49.275864 51.10145 ]
  [51.20403  50.972332 50.293156 50.812843 ... 49.41679  47.980057 48.157036 49.753654]
  [51.6559   50.818684 51.53644  51.319366 ... 48.6491   47.95225  49.674015 51.593887]
  ...
  [50.98786  49.152657 47.675056 49.33832  ... 55.949657 55.929028 56.984894 57.824753]
  [49.870415 45.295303 46.97701  49.443848 ... 57.544163 56.914913 57.45043  58.377846]
  [52.83072  51.094654 48.246395 48.08404  ... 58.606686 57.72443  57.769638 58.370384]
  [54.61412  58.636982 49.55326  48.643322 ... 58.608696 58.234215 58.058414 58.79683 ]]

 [[43.610107 50.864433 50.298016 49.580574 ... 46.173306 46.17163  46.631676 47.37595 ]
  [44.855957 46.073925 48.91847  48.268963 ... 45.693447 45.333603 45.329685 45.155193]
  [45.54885  45.925484 48.5358   47.988792 ... 45.8661   44.3892   44.477097 49.24813 ]
  [46.2189

In [0]:
learn.data = data_mr
fn = data_mr.valid_ds.x.items[0]; fn
img = open_image(fn); img.shape
p,img_hr,b = learn.predict(img)
Image(img_hr).show(figsize=(6,6))

In [0]:
img_ht_np2[0].mean()

0.15231423